# 统计学习导论第九次作业
作业任务：基于用户的协同过滤和基于物品的协同过滤  
surprise库都能实现，伟大，但基于物品的算的太慢了

姓名：欧阳瑞志、班级：统计22、学号：220403102

## 一、数据导入
### 数据来源
Grouplens提供的数据集MovieLens 
### 主题
推荐算法--协同过滤
### 数据地址
https://grouplens.org/datasets/movielens/1m/

In [2]:
####--------------------------------统计学习导论第九次作业（协同过滤）--------------------------------####

from surprise import Dataset
from surprise import Reader
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
import numpy as np
import time

# --------------------------------1、数据导入--------------------------------#
# Grouplens提供的数据集MovieLens 
# 主题：推荐算法--协同过滤
# 地址：https://grouplens.org/datasets/movielens/1m/

data = Dataset.load_builtin('ml-1m')

reader = Reader(line_format='user item rating timestamp', sep='\t')
trainset = data.build_full_trainset()

num_users = trainset.n_users
num_items = trainset.n_items
all_ratings = trainset.all_ratings()

print(f"Number of users: {num_users}")
print(f"Number of items: {num_items}")
print(f"All ratings: {all_ratings}")

first_10_ratings = list(all_ratings)[:10]
print(first_10_ratings)

Number of users: 6040
Number of items: 3706
All ratings: <generator object Trainset.all_ratings at 0x110e152e0>
[(0, 0, 5.0), (0, 1, 3.0), (0, 2, 3.0), (0, 3, 4.0), (0, 4, 5.0), (0, 5, 3.0), (0, 6, 5.0), (0, 7, 5.0), (0, 8, 4.0), (0, 9, 4.0)]


## 二、基于用户的协同过滤
### 2.1 调参
__相似度__：'cosine'，余弦相似度；'pearson'，皮尔逊相似系数；'msd'，均方差；'pearson_baseline'，基于baseline的pearson相似系数  
__k值__：考虑从2到500，这里仅展示最后一步调整k值

In [3]:
# --------------------------------2、基于用户的协同过滤进行推荐--------------------------------#
reader = Reader(rating_scale=(1, 5))

## 调参
start_time = time.time()
# 定义不同的相似度计算方法和k值的组合进行尝试
sim_names = ['cosine', 'pearson', 'msd', 'pearson_baseline']  # 余弦相似度、皮尔逊相关系数、均方差
k_values = [5, 10, 20, 30, 40, 50]

best_mape = float('inf')
best_params = None

# MAPE函数
def mape(true_ratings, predicted_ratings):
    errors = []
    for true_r, pred_r in zip(true_ratings, predicted_ratings):
        if true_r!= 0:
            error = abs((true_r - pred_r) / true_r)
            errors.append(error)
    return np.mean(errors) if errors else 0

# 遍历不同的参数组合
for sim_name in sim_names:
    for k in k_values:
        reader = Reader(rating_scale=(1, 5))

        # 构建基于用户的协同过滤推荐模型
        sim_options = {'name': sim_name, 'user_based': True}  ## user_based设为True为基于用户的协同过滤
        model = KNNBasic(k=k, sim_options=sim_options)  ## 算法也可以调：KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
        model.fit(trainset)

        true_ratings = []
        predicted_ratings = []

        # 计算所有用户的真实评分与预测评分
        for user_id in range(trainset.n_users):
            for item_id, rating in trainset.ur[user_id]:
                prediction = model.predict(user_id, item_id).est
                true_ratings.append(rating)
                predicted_ratings.append(prediction)

        # 计算MAPE
        current_mape = mape(true_ratings, predicted_ratings)
        print(f"使用 {sim_name} 相似度，k={k} 时的 MAPE: {current_mape}")

        # 更新最佳参数组合和最小的MAPE值
        if current_mape < best_mape:
            best_mape = current_mape
            best_params = (sim_name, k)

print(f"最佳参数组合为：相似度计算方法 '{best_params[0]}'，k值 {best_params[1]}，对应的最小MAPE为 {best_mape}")

end_time = time.time()
print(f"基于用户的协同过滤调参耗时: {end_time - start_time} 秒")

print('########调参结束########')

Computing the cosine similarity matrix...
Done computing similarity matrix.
使用 cosine 相似度，k=5 时的 MAPE: 0.38133226143416266
Computing the cosine similarity matrix...
Done computing similarity matrix.
使用 cosine 相似度，k=10 时的 MAPE: 0.38133226143416266
Computing the cosine similarity matrix...
Done computing similarity matrix.
使用 cosine 相似度，k=20 时的 MAPE: 0.38133226143416266
Computing the cosine similarity matrix...
Done computing similarity matrix.
使用 cosine 相似度，k=30 时的 MAPE: 0.38133226143416266
Computing the cosine similarity matrix...
Done computing similarity matrix.
使用 cosine 相似度，k=40 时的 MAPE: 0.38133226143416266
Computing the cosine similarity matrix...
Done computing similarity matrix.
使用 cosine 相似度，k=50 时的 MAPE: 0.38133226143416266
Computing the pearson similarity matrix...
Done computing similarity matrix.
使用 pearson 相似度，k=5 时的 MAPE: 0.38133226143416266
Computing the pearson similarity matrix...
Done computing similarity matrix.
使用 pearson 相似度，k=10 时的 MAPE: 0.38133226143416266
Comput

### 2.2 KNNBasic实现基于用户的协同分析
方法考虑了： KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline，对比MAPE选择KNNBasic

In [4]:
## 最佳参数进模型
sim_options = {'name': 'cosine', 'user_based': True}
model = KNNBasic(k=5, sim_options=sim_options)
model.fit(trainset)

# 推荐用户ID范围设为1到10
for target_user_id in range(1, 11):
    # 获取目标用户已经看过的电影
    watched_movies_by_target_user = trainset.ur[target_user_id]
    watched_movie_ids = [movie_id for (movie_id, _) in watched_movies_by_target_user]

    recommendations = []

    for other_user_id in range(trainset.n_users):
        if other_user_id!= target_user_id:
            similarity = model.sim[target_user_id][other_user_id]
            if similarity > 0:  # 只考虑有正相似度的用户，表示相似用户
                other_user_watched_movies = trainset.ur[other_user_id]
                for movie_id, _ in other_user_watched_movies:
                    if movie_id not in watched_movie_ids:
                        # 预测目标用户对该电影的评分
                        prediction = model.predict(target_user_id, movie_id).est
                        recommendations.append((movie_id, prediction))

    final_recommendations = []
    # 去重
    added_movie_ids = set()
    # 先按照评分从高到低排序
    sorted_recommendations = sorted(recommendations, key=lambda x: x[1], reverse=True)
    # 遍历排序后的推荐结果，取评分排名前十的不同电影
    for movie_id, score in sorted_recommendations:
        if len(final_recommendations) < 10 and movie_id not in added_movie_ids:
            final_recommendations.append((movie_id, score))
            added_movie_ids.add(movie_id)

    print(f"为用户 {target_user_id} 推荐的电影（电影ID，预测评分）：")
    for movie_id, score in final_recommendations:
        print(f"电影ID: {movie_id}, 预测评分: {score}")

Computing the cosine similarity matrix...
Done computing similarity matrix.
为用户 1 推荐的电影（电影ID，预测评分）：
电影ID: 1, 预测评分: 3.581564453029317
电影ID: 2, 预测评分: 3.581564453029317
电影ID: 3, 预测评分: 3.581564453029317
电影ID: 4, 预测评分: 3.581564453029317
电影ID: 5, 预测评分: 3.581564453029317
电影ID: 6, 预测评分: 3.581564453029317
电影ID: 7, 预测评分: 3.581564453029317
电影ID: 8, 预测评分: 3.581564453029317
电影ID: 9, 预测评分: 3.581564453029317
电影ID: 10, 预测评分: 3.581564453029317
为用户 2 推荐的电影（电影ID，预测评分）：
电影ID: 0, 预测评分: 3.581564453029317
电影ID: 1, 预测评分: 3.581564453029317
电影ID: 2, 预测评分: 3.581564453029317
电影ID: 3, 预测评分: 3.581564453029317
电影ID: 6, 预测评分: 3.581564453029317
电影ID: 7, 预测评分: 3.581564453029317
电影ID: 8, 预测评分: 3.581564453029317
电影ID: 9, 预测评分: 3.581564453029317
电影ID: 10, 预测评分: 3.581564453029317
电影ID: 11, 预测评分: 3.581564453029317
为用户 3 推荐的电影（电影ID，预测评分）：
电影ID: 0, 预测评分: 3.581564453029317
电影ID: 1, 预测评分: 3.581564453029317
电影ID: 2, 预测评分: 3.581564453029317
电影ID: 3, 预测评分: 3.581564453029317
电影ID: 4, 预测评分: 3.581564453029317
电影ID: 5, 预测评分: 3.5815644

## 三、基于物品的协同分析
### 3.1 调参
__相似度__：'cosine'，余弦相似度；'pearson'，皮尔逊相似系数；'msd'，均方差；'pearson_baseline'，基于baseline的pearson相似系数  
__k值__：考虑从2到500，这里仅展示最后一步调整k值

In [5]:
# --------------------------------3、基于物品的协同过滤进行推荐--------------------------------#

## 调参
start_time = time.time()

sim_names_item = ['cosine', 'pearson', 'msd', 'pearson_baseline']  # 余弦相似度、皮尔逊相关系数、均方差
k_values_item = [5, 10, 20, 30, 40, 50]

best_mape_item = float('inf')
best_params_item = None

for sim_name in sim_names_item:
    for k in k_values_item:

        # 构建基于物品的协同过滤推荐模型
        sim_options = {'name': sim_name, 'user_based': False}  # 设置user_based为False，实现基于物品的协同过滤
        model = KNNWithZScore(k=k, sim_options=sim_options)
        model.fit(trainset)

        true_ratings = []
        predicted_ratings = []

        # 计算真实评分与预测评分
        for user_id in range(trainset.n_users):
            for item_id, rating in trainset.ur[user_id]:
                prediction = model.predict(user_id, item_id).est
                true_ratings.append(rating)
                predicted_ratings.append(prediction)

        # 计算MAPE
        current_mape = mape(true_ratings, predicted_ratings)
        print(f"使用 {sim_name} 相似度，k={k} 时的 MAPE（基于物品的协同过滤）: {current_mape}")

        # 更新最佳参数组合和最小的MAPE值（基于物品的协同过滤）
        if current_mape < best_mape_item:
            best_mape_item = current_mape
            best_params_item = (sim_name, k)

print(f"基于物品的协同过滤最佳参数组合为：相似度计算方法 '{best_params_item[0]}'，k值 {best_params_item[1]}，对应的最小MAPE为 {best_mape_item}")

end_time = time.time()
print(f"基于物品的协同过滤调参耗时: {end_time - start_time} 秒")

print('########调参结束########')

Computing the cosine similarity matrix...
Done computing similarity matrix.
使用 cosine 相似度，k=5 时的 MAPE（基于物品的协同过滤）: 0.38133226143416266
Computing the cosine similarity matrix...
Done computing similarity matrix.
使用 cosine 相似度，k=10 时的 MAPE（基于物品的协同过滤）: 0.38133226143416266
Computing the cosine similarity matrix...
Done computing similarity matrix.
使用 cosine 相似度，k=20 时的 MAPE（基于物品的协同过滤）: 0.38133226143416266
Computing the cosine similarity matrix...
Done computing similarity matrix.
使用 cosine 相似度，k=30 时的 MAPE（基于物品的协同过滤）: 0.38133226143416266
Computing the cosine similarity matrix...
Done computing similarity matrix.
使用 cosine 相似度，k=40 时的 MAPE（基于物品的协同过滤）: 0.38133226143416266
Computing the cosine similarity matrix...
Done computing similarity matrix.
使用 cosine 相似度，k=50 时的 MAPE（基于物品的协同过滤）: 0.38133226143416266
Computing the pearson similarity matrix...
Done computing similarity matrix.
使用 pearson 相似度，k=5 时的 MAPE（基于物品的协同过滤）: 0.38133226143416266
Computing the pearson similarity matrix...
Done computi

### 3.2 KNNWithZScore实现基于物品的协同分析
方法考虑了： KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline，对比MAPE选择KNNBasic   
这一步由于要便利所有ID，这一步需要3h    
为了观测计算进度，输出了每次计算结果，共3700+条

In [6]:
# 最佳参数进模型
sim_options_item = {'name': best_params_item[0], 'user_based': False, 'k': best_params_item[1]}
model_item = KNNWithZScore(sim_options=sim_options_item)
model_item.fit(trainset)

start_time = time.time()

for target_user_id in [1]:
    watched_items_ratings = []
    for user_id, item_id, rating in trainset.all_ratings():
        if item_id in [movie_id for (movie_id, _) in trainset.ur[target_user_id]]:
            watched_items_ratings.append((item_id, rating))

    watched_movie_ids_item = [movie_id for movie_id, _ in watched_items_ratings]

    recommendations_item = []

    for item_id in range(trainset.n_items):
        if item_id not in watched_movie_ids_item:
            similarity = model_item.sim[item_id]
            print(f"Item ID: {item_id}, Similarity values: {similarity}")
            for other_item_id, similarity_value in enumerate(similarity):
                if other_item_id in watched_movie_ids_item and similarity_value > 0:

                    prediction = model_item.predict(target_user_id, item_id).est
                    recommendations_item.append((item_id, prediction))

    final_recommendations_item = []
    added_movie_ids_item = set()
    # 先按照评分从高到低排序
    sorted_recommendations_item = sorted(recommendations_item, key=lambda x: x[1], reverse=True)
    # 遍历排序后的推荐结果，取评分排名前十的不同电影
    for movie_id, score in sorted_recommendations_item:
        if len(final_recommendations_item) < 10 and movie_id not in added_movie_ids_item:
            final_recommendations_item.append((movie_id, score))
            added_movie_ids_item.add(movie_id)

    print(f"基于物品的协同过滤为用户 {target_user_id} 推荐的电影（电影ID，预测评分）：")
    for movie_id, score in final_recommendations_item:
        print(f"电影ID: {movie_id}, 预测评分: {score}")

end_time = time.time()
print(f"基于物品的协同过滤最佳参数计算耗时: {end_time - start_time} 秒")

Computing the cosine similarity matrix...
Done computing similarity matrix.
Item ID: 1, Similarity values: [0.95556011 1.         0.93827998 ... 0.         0.         0.        ]
Item ID: 2, Similarity values: [0.96639164 0.93827998 1.         ... 0.         0.         0.        ]
Item ID: 3, Similarity values: [0.96067846 0.93029121 0.96632659 ... 0.         0.         0.        ]
Item ID: 4, Similarity values: [0.96369964 0.95890738 0.96621826 ... 0.         0.         0.        ]
Item ID: 5, Similarity values: [0.96810869 0.94726375 0.96514344 ... 0.         1.         1.        ]
Item ID: 6, Similarity values: [0.96678531 0.94502466 0.95972701 ... 0.         0.         0.        ]
Item ID: 7, Similarity values: [0.96782115 0.93832889 0.96083452 ... 0.         0.         0.        ]
Item ID: 8, Similarity values: [0.96171441 0.94813431 0.96777411 ... 0.         0.         0.        ]
Item ID: 9, Similarity values: [0.96931821 0.94987369 0.97296714 ... 0.         1.         0.       